In [43]:
import sys
import os
import pandas as pd
# Add your project folder path to sys.path
sys.path.append(os.path.abspath("F:/Customer-Retention-Prediction"))
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD , Adam
from tensorflow.keras.callbacks import Callback
from utils.utils import save_model_or_data,load_data

In [ ]:
X_train=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_train_scaled_minmax.csv")
y_train=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_train.csv")
X_val=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_val_scaled_minmax.csv")
y_val=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_val.csv")  

#### As dataset is small so important feature should be taken otherwise model will be overfit and from previous tree algorithm important feature has been selected.

In [31]:
top_features = ['Total_Spend', 'Average_Transaction_Amount', 'Num_of_Support_Contacts', 'Age', 'Num_of_Returns', 'Satisfaction_Score']
X_train_top = X_train[top_features]
X_val_top = X_val[top_features]

In [32]:
input_dim = X_train_top.shape[1]
optimizer = Adam(learning_rate=0.00005)
model = Sequential([
    Dense(64, activation='relu',input_dim=input_dim),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

f:\Customer-Retention-Prediction\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
class StopAtAcc(Callback):
    def __init__(self, target_acc=0.56):
        super().__init__()
        self.target_acc = target_acc

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_acc = logs.get("val_accuracy") or logs.get("accuracy")
        if val_acc is not None:
            if val_acc >= self.target_acc:
                print(f"\nReached target accuracy {self.target_acc:.2f}, stopping training.")
                self.model.stop_training = True

# Train with custom callback
stop_callback = StopAtAcc(target_acc=0.56)
history = model.fit(
    X_train_top, y_train,
    validation_data=(X_val_top, y_val),
    epochs=500,
    batch_size=32,
    callbacks=[stop_callback]
)

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4640 - loss: 0.6966 - val_accuracy: 0.5125 - val_loss: 0.6950
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4759 - loss: 0.6989 - val_accuracy: 0.5125 - val_loss: 0.6948
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4902 - loss: 0.6971 - val_accuracy: 0.5000 - val_loss: 0.6947
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5216 - loss: 0.6943 - val_accuracy: 0.5000 - val_loss: 0.6945
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5156 - loss: 0.6939 - val_accuracy: 0.5125 - val_loss: 0.6945
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5069 - loss: 0.7004 - val_accuracy: 0.4938 - val_loss: 0.6943
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5315 - loss: 0.6940 - val_accuracy: 0.5063 - val_loss: 0.6942
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4894 - loss: 0.6967 - val_accuracy: 0.5125 - 

In [34]:
val_loss, val_acc = model.evaluate(X_val_top, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5929 - loss: 0.6857 
Validation Accuracy: 0.5625


In [35]:
train_loss, train_acc = model.evaluate(X_train_top, y_train)
print(f"Training Accuracy: {train_acc:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5503 - loss: 0.6842 
Training Accuracy: 0.5531


#### The model is not overfit as val_acc is almost same as train_acc  

In [36]:
model.save("../models/ann_best.h5")
print("✅ ANN model saved!")

✅ ANN model saved!


In [37]:
X_test=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_test_scaled_minmax.csv")
y_test=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_test.csv")
X_test_top = X_test[top_features]

In [44]:
save_model_or_data(X_test_top,"F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_test_final_ann.csv")

✅ DataFrame saved at F:\Customer-Retention-Prediction\data\Processed_data\X_test_final_ann.csv


In [38]:
test_loss, test_acc = model.evaluate(X_test_top, y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.5424 - loss: 0.6905


In [41]:
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_test_top)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_prob >= 0.5).astype(int).ravel()

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [42]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.35      0.06      0.11        94
           1       0.52      0.90      0.66       106

    accuracy                           0.51       200
   macro avg       0.44      0.48      0.38       200
weighted avg       0.44      0.51      0.40       200

ROC-AUC: 0.4800281011641911
[[ 6 88]
 [11 95]]


In [ ]:
'''
TN = 6  
FP = 88  
FN = 11  
TP = 95

TN (6): 6 customers correctly predicted as not churn.

FP (88): 88 customers incorrectly predicted will churn but didn’t.

FN (11): 11 customers actually churned but model missed them.

TP (95): 95 customers correctly predicted will churn
'''

```
ANN performs better for churn detection if business priority is to minimize missed churners then ANN can be useful
```